In [1]:
import tf_keras as keras
from transformers import pipeline, set_seed


In [3]:
pip install tf-keras --quiet


Note: you may need to restart the kernel to use updated packages.


In [2]:
# Install necessary libraries
#!pip install tf-keras torch transformers --quiet

import numpy as np
import tf_keras as keras
from tf_keras.preprocessing.text import Tokenizer
from tf_keras.preprocessing.sequence import pad_sequences
from tf_keras.models import Sequential
from tf_keras.layers import Embedding, LSTM, Dense
from transformers import pipeline, set_seed

# Sample corpus
corpus = [
    "Artificial intelligence is transforming industries.",
    "Natural language processing is a field of AI.",
    "Machine learning helps computers learn from data.",
    "Neural networks are used in deep learning.",
    "AI is the future of technology and innovation."
]

# Tokenization and preprocessing
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_seq = token_list[:i+1]
        input_sequences.append(n_gram_seq)

max_seq_len = max(len(x) for x in input_sequences)
input_sequences = pad_sequences(input_sequences, maxlen=max_seq_len, padding='pre')
input_sequences = np.array(input_sequences)
X, y = input_sequences[:, :-1], input_sequences[:, -1]
y = keras.utils.to_categorical(y, num_classes=total_words)

# Build and train LSTM model
model = Sequential([
    Embedding(total_words, 64, input_length=max_seq_len-1),
    LSTM(100),
    Dense(total_words, activation='softmax')
])
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=1)

# Function to generate text using LSTM
def generate_text_lstm(seed_text, next_words=20):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_len-1, padding='pre')
        predicted = np.argmax(model.predict(token_list, verbose=0), axis=-1)
        output_word = ''
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += ' ' + output_word
    return seed_text

print("LSTM Output:\n", generate_text_lstm("Artificial intelligence"))

# GPT-2 text generation using PyTorch
generator = pipeline("text-generation", model="gpt2", framework="pt")
set_seed(42)

def generate_text_gpt(prompt, max_len=100):
    result = generator(prompt, max_length=max_len, num_return_sequences=1)
    return result[0]['generated_text']

print("\nGPT-2 Output:\n", generate_text_gpt("The impact of artificial intelligence on education"))




Epoch 1/200


1/1 [==============================] - 8s 8s/step - loss: 3.4359 - accuracy: 0.0333
Epoch 2/200
1/1 [==============================] - 0s 18ms/step - loss: 3.4307 - accuracy: 0.0333
Epoch 3/200
1/1 [==============================] - 0s 23ms/step - loss: 3.4255 - accuracy: 0.1333
Epoch 4/200
1/1 [==============================] - 0s 12ms/step - loss: 3.4202 - accuracy: 0.1333
Epoch 5/200
1/1 [==============================] - 0s 18ms/step - loss: 3.4147 - accuracy: 0.1333
Epoch 6/200
1/1 [==============================] - 0s 18ms/step - loss: 3.4089 - accuracy: 0.1333
Epoch 7/200
1/1 [==============================] - 0s 23ms/step - loss: 3.4028 - accuracy: 0.1333
Epoch 8/200
1/1 [==============================] - 0s 15ms/step - loss: 3.3961 - accuracy: 0.1333
Epoch 9/200
1/1 [==============================] - 0s 20ms/step - loss: 3.3890 - accuracy: 0.1333
Epoch 10/200
1/1 [==============================] - 0s 19ms/step - loss: 3.3811 - accuracy: 0.1000
Epoch 11/200
1/1 

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

C:\Users\SHUBHAM\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\SHUBHAM\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

...

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



GPT-2 Output:
 The impact of artificial intelligence on education is not just a technical one in terms of improving our science-based understanding of how the mind works. Its impact on the educational system is not just an academic issue in terms of having a more robust system of education. It is a social issue.

I have had people from a range of different backgrounds take the role of President Obama as their lead advisor on such issues as education, health care, and climate change. I have had students from different backgrounds speaking
